In [ ]:
import pandas as pd
train_concat = pd.read_csv("../data_preprocessed/train_concat.csv.gz")
from numpy.random import seed
seed(1)

In [ ]:
train_concat = train_concat.drop(['Unnamed: 0', 'sWeight', 'kinWeight'], axis=1)

In [ ]:
train_features = ['ncl[0]', 'ncl[1]', 'ncl[2]', 'ncl[3]', 'avg_cs[0]', 'avg_cs[1]',
       'avg_cs[2]', 'avg_cs[3]', 'ndof', 'MatchedHit_X[0]', 'MatchedHit_X[1]',
       'MatchedHit_X[2]', 'MatchedHit_X[3]', 'MatchedHit_Y[0]',
       'MatchedHit_Y[1]', 'MatchedHit_Y[2]', 'MatchedHit_Y[3]',
       'MatchedHit_Z[0]', 'MatchedHit_Z[1]', 'MatchedHit_Z[2]',
       'MatchedHit_Z[3]', 'MatchedHit_DX[0]', 'MatchedHit_DX[1]',
       'MatchedHit_DX[2]', 'MatchedHit_DX[3]', 'MatchedHit_DY[0]',
       'MatchedHit_DY[1]', 'MatchedHit_DY[2]', 'MatchedHit_DY[3]',
       'MatchedHit_DZ[0]', 'MatchedHit_DZ[1]', 'MatchedHit_DZ[2]',
       'MatchedHit_DZ[3]', 'MatchedHit_T[0]', 'MatchedHit_T[1]',
       'MatchedHit_T[2]', 'MatchedHit_T[3]', 'MatchedHit_DT[0]',
       'MatchedHit_DT[1]', 'MatchedHit_DT[2]', 'MatchedHit_DT[3]',
       'Lextra_X[0]', 'Lextra_X[1]', 'Lextra_X[2]', 'Lextra_X[3]',
       'Lextra_Y[0]', 'Lextra_Y[1]', 'Lextra_Y[2]', 'Lextra_Y[3]', 'NShared',
       'Mextra_DX2[0]', 'Mextra_DX2[1]', 'Mextra_DX2[2]', 'Mextra_DX2[3]',
       'Mextra_DY2[0]', 'Mextra_DY2[1]', 'Mextra_DY2[2]', 'Mextra_DY2[3]',
       'FOI_hits_N', 'PT', 'P', '0', '1', '2', '3', '4',
       '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17',
       '18', '19', '20', '21', '22', '23']
cat_features = [ 'MatchedHit_TYPE[0]', 'MatchedHit_TYPE[1]', 'MatchedHit_TYPE[2]', 'MatchedHit_TYPE[3]']

In [ ]:
train_concat['PT_P'] = train_concat.P - train_concat.PT
train_features.append('PT_P')
train_concat['PT_P/P'] = train_concat.PT_P / train_concat.P
train_features.append('PT_P/P')

In [ ]:
def has_missing(data):
    missing = False
    values = []
    for i in data:
        if i == -9999:
            missing = True
        else:
            missing = False
        values.append(missing)
    return values

In [ ]:
added_features = []
nans = []
for i in ['MatchedHit_X[0]', 'MatchedHit_X[1]', 'MatchedHit_X[2]', 'MatchedHit_X[3]', 
          'MatchedHit_Y[0]', 'MatchedHit_Y[1]', 'MatchedHit_Y[2]', 'MatchedHit_Y[3]']:
    train_concat['abs_%s' %i] = abs(train_concat['%s' %i])
    added_features.append('abs_%s' %i)
    nans.append('nan_%s' %i)
    train_concat['nan_%s' %i] = has_missing(train_concat['%s' %i])

In [ ]:
train_concat = pd.get_dummies(train_concat, columns = cat_features)

In [ ]:
dummies = ['MatchedHit_TYPE[0]_1',
       'MatchedHit_TYPE[0]_2', 'MatchedHit_TYPE[1]_1', 'MatchedHit_TYPE[1]_2',
       'MatchedHit_TYPE[2]_0', 'MatchedHit_TYPE[2]_1', 'MatchedHit_TYPE[2]_2',
       'MatchedHit_TYPE[3]_0', 'MatchedHit_TYPE[3]_1', 'MatchedHit_TYPE[3]_2']

## Private

In [ ]:
test_concat = pd.read_csv("../data_preprocessed/private_concat.csv.gz")

In [ ]:
test_concat['PT_P'] = test_concat.P - test_concat.PT
test_concat['PT_P/P'] = test_concat.PT_P/test_concat.P

In [ ]:
for i in ['MatchedHit_X[0]', 'MatchedHit_X[1]', 'MatchedHit_X[2]', 'MatchedHit_X[3]', 
          'MatchedHit_Y[0]', 'MatchedHit_Y[1]', 'MatchedHit_Y[2]', 'MatchedHit_Y[3]']:
    test_concat['abs_%s' %i] = abs(test_concat['%s' %i])
    test_concat['nan_%s' %i] = has_missing(test_concat['%s' %i])

In [ ]:
test_concat = pd.get_dummies(test_concat, columns = cat_features)

In [ ]:
dummies_test = ['MatchedHit_TYPE[0]_1.0', 'MatchedHit_TYPE[0]_2.0',
       'MatchedHit_TYPE[1]_1.0', 'MatchedHit_TYPE[1]_2.0',
       'MatchedHit_TYPE[2]_0.0', 'MatchedHit_TYPE[2]_1.0',
       'MatchedHit_TYPE[2]_2.0', 'MatchedHit_TYPE[3]_0.0',
       'MatchedHit_TYPE[3]_1.0', 'MatchedHit_TYPE[3]_2.0']

## KFold

In [ ]:
from sklearn.model_selection import KFold # import KFold
kf = KFold(n_splits = 5, random_state = 41, shuffle = True)

## Prediction

In [ ]:
import lightgbm as lgb
validation_predictions = []
test_predictions = []

In [ ]:
del(KFold, cat_features, has_missing, i, pd)

In [ ]:
for train_index, test_index in kf.split(train_concat):
    print(train_index, test_index)
    X_train, X_val = train_concat.loc[train_index], train_concat.loc[test_index]
    y_train, y_val = train_concat.label[train_index], train_concat.label[test_index]
    train_data = lgb.Dataset(X_train[train_features+added_features+dummies+nans], 
                         label = y_train, weight=X_train.weight, 
                         feature_name= train_features+added_features+dummies+nans, 
                         categorical_feature=dummies+nans)
    param = {'random_state':42, 'max_depth':9, 'objective':'binary', 'learning_rate':0.2,'num_leaves':128,'min_data_in_leaf':15, 'num_iterations':90}
    model = lgb.train(param, train_data)                        
    prediction = model.predict(X_val[train_features+added_features+dummies+nans])
    validation_predictions.append(prediction)
    prediction_test = model.predict(test_concat[train_features+added_features+dummies_test+nans])
    test_predictions.append(list(prediction_test))

In [ ]:
validations = []
for sublist in validation_predictions:
    for item in sublist:
        validations.append(item)

In [ ]:
test=[]
for i in range(len(test_predictions[0])):
    sum=0
    for j in range(5):
        sum+=test_predictions[j][i]
    test.append(sum / 5)

In [ ]:
import numpy as np
np.savetxt("../predictions/lgbm_dummies_val", validations)
np.savetxt("../predictions/lgbm_dummies_private", test)